In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

file_path = 'train-00000-of-00039.parquet'

df = pd.read_parquet(file_path)

In [ ]:
if ( df.loc[df['id'] == 1, 'descendants'].any() > 0):
    print( df.loc[df['id'] == 1, 'descendants'] > 0)
    x = df.loc[df['id'] == 1, 'descendants']
    print( x + 2 )

In [ ]:
df = df.head(100000)


In [ ]:
df['html'] = 'null'  
df['comments'] = 'null'  

# Attempt 1

In [ ]:
def parent_to_text(parent_id):
    visited_ids = set()

    def traverse_and_collect(parent_id):
        parent = df[df['id'] == parent_id]
        if parent.empty:
            return #"Parent not found"
        
        visited_ids.add(parent_id)  # add parent to visited ids

        kids = parent['kids'].values[0]
        if kids is not None:
            if not all(kid in df['id'].values for kid in kids):  
                return #"We can't create the story" if any kid is missing
            
            current_text = parent['text'].values[0] if parent['text'].values[0] is not None else ""
            for kid_id in kids:
                child_text = traverse_and_collect(kid_id)
                if child_text == "We can't create the story":
                    return #"We can't create the story" abort if any child's subtree is incomplete
                current_text += "\n" + child_text
        else:
            current_text = parent['text'].values[0] if parent['text'].values[0] is not None else ""

        return current_text

    final_text = traverse_and_collect(parent_id)
    if final_text != "We can't create the story" and final_text != "Parent not found":
        # remove all visited ids except the root parent
        global df
        df = df[~df['id'].isin(visited_ids - {parent_id})]

    return final_text

In [ ]:
for ind in df["id"]:
    if df.loc[df['id'] == ind, 'kids'].notnull().any():
        df.loc[df['id'] == ind, 'comments'] = parent_to_text(ind)
    
    if df.loc[df['id'] == ind, 'url'].notnull().any():
        url = df.loc[df['id'] == ind, 'url'].values[0]
        try:

            response = requests.get(url, timeout=5)
            response.raise_for_status()  # raise HTTPError for bad responses
            
            soup = BeautifulSoup(response.text, 'html.parser')
            plain_text = soup.get_text()
            
            df.loc[df['id'] == ind, 'html'] = plain_text
        except (requests.RequestException, Exception) as e:
            #print(f"Error fetching content for URL {url}: {e}")
            df = df[df['id'] != ind]
    else:
        # ff there is no URL and it hasn't been removed previously remove it
        df = df[df['id'] != ind]

df.to_parquet('processed_0.parquet', engine='pyarrow', index=False)


In [ ]:
print(df.head(20))

In [ ]:
print(df.dtypes)

# Save

In [99]:
print(ind)

NameError: name 'ind' is not defined

In [ ]:
df.to_parquet('processed_01.parquet', engine='pyarrow', index=False)


# Absolete

In [ ]:
parents = []
desc = []


print(df.shape[0])

for i in range(df.shape[0]):
    if (df['descendants'].iloc[i] > 0):
        parents.append(i)
        desc.append(df['descendants'].iloc[i])

print("Parents:", parents)
print("Descendants:", desc)
print(len(parents))
print(len(desc))

In [ ]:
news = []
for i in range(df.shape[0]):
    if (pd.isna(df.at[i, 'parent']) or pd.isna(df.at[i, 'url'])):
        news.append(i)

In [ ]:
import requests

url = "http://www.paulgraham.com/startuplessons.html" 

try:
    response = requests.get(url)
    response.raise_for_status()  # raise an error
    my_html = response.text  # store HTML content
    print(my_html)  
except requests.RequestException as e:
    print(f"Error fetching the URL: {e}")
    my_html = None  


In [ ]:
import pandas as pd

data = {'Column1': [1, 2, 3], 'Column2': ['A', 'B', 'C']}
df = pd.DataFrame(data)

#df.to_parquet('example.parquet', engine='pyarrow', index=False)


# Threading

In [ ]:
def parent_to_text(parent_id):
    #visited_ids = set()

    def traverse_and_collect(parent_id):
        print(f"Parent: {parent_id}")
        parent = df[df['id'] == parent_id]
        if parent.empty:
            return #"Parent not found"
        
        #visited_ids.add(parent_id)  # add parent to visited ids

        kids = parent['kids'].values[0]
        if kids is not None:
            if not all(kid in df['id'].values for kid in kids):  
                return #"We can't create the story" if any kid is missing
            
            current_text = parent['text'].values[0] if parent['text'].values[0] is not None else ""
            for kid_id in kids:
                child_text = traverse_and_collect(kid_id)
                if child_text == "We can't create the story":
                    return #"We can't create the story" abort if any child's subtree is incomplete
                current_text += ". " + child_text
        else:
            current_text = parent['text'].values[0] if parent['text'].values[0] is not None else ""

        return current_text

    final_text = traverse_and_collect(parent_id)
    #if final_text != "We can't create the story" and final_text != "Parent not found":
        # remove all visited ids except the root parent
        #global df
        #df = df[~df['id'].isin(visited_ids - {parent_id})]

    return final_text

In [ ]:
for ind in df["id"]:
    if df.loc[df['id'] == ind, 'descendants'] > 0:
        df.loc[df['id'] == ind, 'comments'] = parent_to_text(ind)
    
    if df.loc[df['id'] == ind, 'url'].notnull().any():
        url = df.loc[df['id'] == ind, 'url'].values[0]
        try:

            response = requests.get(url, timeout=5)
            response.raise_for_status()  # raise HTTPError for bad responses
            
            soup = BeautifulSoup(response.text, 'html.parser')
            plain_text = soup.get_text()
            
            df.loc[df['id'] == ind, 'html'] = plain_text
        except (requests.RequestException, Exception) as e:
            #print(f"Error fetching content for URL {url}: {e}")
            df = df[df['id'] != ind]
    else:
        # ff there is no URL and it hasn't been removed previously remove it
        df = df[df['id'] != ind]


In [ ]:
df.to_parquet('processed_0.parquet', engine='pyarrow', index=False)

# More Threadding

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import requests
from bs4 import BeautifulSoup

# Lock to ensure thread-safe DataFrame modifications
df_lock = threading.Lock()

def fetch_url_and_process(ind):
    global df
    with df_lock:
        url = df.loc[df['id'] == ind, 'url'].values[0]

    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        plain_text = soup.get_text()

        # Thread-safe update of the DataFrame
        with df_lock:
            df.loc[df['id'] == ind, 'html'] = plain_text
    except (requests.RequestException, Exception) as e:
        # Thread-safe removal of the row
        with df_lock:
            df = df[df['id'] != ind]

def process_descendants(ind):
    global df
    with df_lock:
        #print( df.loc[df['id'] == ind, 'descendants'])
        if df.loc[df['id'] == ind, 'descendants'].any() > 0: # JOKES ON YOU DE AICI NU MAI FAC NIMIC
            df.loc[df['id'] == ind, 'comments'] = parent_to_text(ind) # NIMIC SCRIS IN COMMENTS

# Main function to manage threading
def process_dataframe(df):
    with ThreadPoolExecutor(max_workers=20) as executor:
        futures = []
        for ind in df["id"]:
            # Submit tasks for descendant processing
            futures.append(executor.submit(process_descendants, ind))

            # Submit tasks for URL fetching
            if df.loc[df['id'] == ind, 'url'].notnull().any():
                futures.append(executor.submit(fetch_url_and_process, ind))
            else:
                # Thread-safe removal for rows without a URL
                with df_lock:
                    df = df[df['id'] != ind]

        # Wait for all threads to complete
        for future in as_completed(futures):
            try:
                future.result()  # Retrieve results (if any) and raise exceptions
            except Exception as e:
                print(f"Error in thread: {e}")


In [ ]:
process_dataframe(df)

In [ ]:
print(df[["type", "descendants" , "comments"]])

In [ ]:
df.to_parquet('processed_0_threadding.parquet', engine='pyarrow', index=False)

# Working Iteration?

In [92]:
import pandas as pd
import numpy as np
import requests
import threading
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed

file_path = 'train-00000-of-00039.parquet'

df = pd.read_parquet(file_path)

In [93]:
df['html'] = 'null'  
df['comments'] = 'null'  

In [94]:
def parent_to_text(parent_id):
    #visited_ids = set()

    def traverse_and_collect(parent_id):
        parent = df[df['id'] == parent_id]
        if parent.empty:
            return #"Parent not found"
        
        #visited_ids.add(parent_id)  # add parent to visited ids

        kids = parent['kids'].values[0]
        if kids is not None:
            if not all(kid in df['id'].values for kid in kids):  
                return #"We can't create the story" if any kid is missing
            
            current_text = parent['text'].values[0] if parent['text'].values[0] is not None else ""
            for kid_id in kids:
                child_text = traverse_and_collect(kid_id)
                if child_text == "We can't create the story":
                    return #"We can't create the story" abort if any child's subtree is incomplete
                current_text += ". " + child_text
        else:
            current_text = parent['text'].values[0] if parent['text'].values[0] is not None else ""

        return current_text

    final_text = traverse_and_collect(parent_id)
    #if final_text != "We can't create the story" and final_text != "Parent not found":
        # remove all visited ids except the root parent
        #global df
        #df = df[~df['id'].isin(visited_ids - {parent_id})]

    return final_text

In [95]:
# Lock to ensure thread-safe DataFrame modifications
df_lock = threading.Lock()

def fetch_url_and_process(ind):
    global df
    with df_lock:
        url = df.loc[df['id'] == ind, 'url'].values[0]

    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        plain_text = soup.get_text()

        # Thread-safe update of the DataFrame
        with df_lock:
            df.loc[df['id'] == ind, 'html'] = plain_text
    except (requests.RequestException, Exception) as e:
        # Thread-safe removal of the row
        with df_lock:
            df = df[df['id'] != ind]

def process_descendants(ind):
    global df
    with df_lock:
        #print( df.loc[df['id'] == ind, 'descendants'])
        if df.loc[df['id'] == ind, 'descendants'].any() > 0: # JOKES ON YOU DE AICI NU MAI FAC NIMIC
            df.loc[df['id'] == ind, 'comments'] = parent_to_text(ind) # NIMIC SCRIS IN COMMENTS

# Main function to manage threading
def process_dataframe(df):
    with ThreadPoolExecutor(max_workers=20) as executor:
        futures = []
        for ind in df["id"]:
            # Submit tasks for descendant processing
            futures.append(executor.submit(process_descendants, ind))

            # Submit tasks for URL fetching
            if df.loc[df['id'] == ind, 'url'].notnull().any():
                futures.append(executor.submit(fetch_url_and_process, ind))
            else:
                # Thread-safe removal for rows without a URL
                with df_lock:
                    df = df[df['id'] != ind]

        # Wait for all threads to complete
        for future in as_completed(futures):
            try:
                future.result()  # Retrieve results (if any) and raise exceptions
            except Exception as e:
                print(f"Error in thread: {e}")


In [96]:
process_dataframe(df)

C:\Users\Ardeleanu Gabriel\AppData\Local\Temp\ipykernel_6276\2273599182.py:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(response.text, 'html.parser')
C:\Users\Ardeleanu Gabriel\AppData\Local\Temp\ipykernel_6276\2273599182.py:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(response.text, 'html.parser')
C:\Users\Ardeleanu Gabriel\AppData\Local\Temp\ipykernel_6276\2273599182.py:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(response.text, 'html.parser')
C:\Users\Ardeleanu Gabriel\AppData\Local\Temp\ipykernel_6276\2273599182.py:13: MarkupResemblesLocatorWarning: The input looks more li

KeyboardInterrupt: 

In [98]:
df.to_parquet('processed_0_threadding_2054_35.parquet', engine='pyarrow', index=False)

In [100]:
print(df.head(5))

   id   type          by                time  \
0   0   None        None                 NaT   
1   1  story          pg 2006-10-09 18:21:51   
2   2  story     phyllis 2006-10-09 18:30:28   
5   5  story      perler 2006-10-09 18:51:04   
8   8  story  frobnicate 2006-10-09 19:17:39   

                                               title  text  \
0                                               None  None   
1                                       Y Combinator  None   
2                      A Student's Guide to Startups  None   
5  Google, YouTube acquisition announcement could...  None   
8                         LikeBetter featured by BBC  None   

                                                 url  score  parent  \
0                                               None    NaN     NaN   
1                             http://ycombinator.com   57.0     NaN   
2                 http://www.paulgraham.com/mit.html   16.0     NaN   
5  http://www.techcrunch.com/2006/10/09/google-yo...  